In [1]:
from docx import *
import pandas as pd
import re
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory

factory = StemmerFactory()
stemmer = factory.create_stemmer()

In [2]:
# read file
# document = Document("Preprocessing/[3]Dictionary IND-ENG-139-161.docx")
document = Document("Converted/pdf2go-execution.docx")

In [3]:
for paragraph in document.paragraphs:
    
    
    # first line indent (segitiga atas di ruler)
    first = paragraph.paragraph_format.first_line_indent
    
    # left indent (segitiga bawah di ruler)
    left = paragraph.paragraph_format.left_indent
    
    print(first,left,paragraph.text+"\n")

None None A

None 0 

None 0 

None 0 

None 0 

None None 

-152400 227965 a and A I /a/ the ¤rst letter of the Latin alphabet used for writing Indonesian.

-152400 227965 a II (abbr) are (a unit of square measure in the metric system, equal to 100 square meters or about 119.6 square yards).

None 75565 a III (abbr) ampere.

None 75565 a IV ahh (hesitation).

-152400 227965 A V (in music) the sixth tone/note in the ascending scale of C ma<jor.

None 75565 A VI car license plate for Banten.

None 75565 à and @ at.

None 75565 – Rp 1.000,- at Rp 1,000.00.

None 75565 A ’45 [Angkatan ’45] the Generation of 1945.

None 75565 AA car license plate for Kedu (Magelang).

None 75565 aah ah.

None 75565 –, itu kan proyék swasta.

None 75565 Ah, that’s a private project.

None 75565 aala and a’ala dynasty; > RAJAKULA.

None 75565 – Tang Tang dynasty.

None 75565 aam (A) general.

None 75565 Rois – General Chairman (of the NU); > AM.

None 75565 aau ow! ouch!

None 75565 ab I (A ob) father.

None

None 76200 abaya long black coat worn by women on the pilgrimage to Mecca.

None 152400 berabaya to wear such a coat.

None 76200 abc and ABC /a­bé­cé/ alphabet, the abc’s.

None 76200 tidak tahu sama sekali

None None – not to have the slightest idea (about something).

None 76200 abd > ABDI.

None 76200 abdas (Pers) (Muslim) ritual ablution before prayer.

None 152400 berabdas to make one’s ritual ablution before prayer.

-153035 228600 abdi (A) 1 (bought) slave.

-153035 228600 2 servant, (domestic) helper.

-153035 228600 – dalam/ dalem (Jv) royal servant in the kraton.

-153035 228600 – dalem jajar (Jv) the lowest rank in the kraton administrative structure.

-153035 228600 – dalem putih (Jv) royal servant in charge of religious affairs.

-153035 228600 –.dalem- isme serfdom.

-153035 228600 – masyarakat public servant.

-153035 228600 – negara servant of the state.

-153035 228600 – rakyat public servant.

None 152400 berabdi to be a servant (of ).

None 152400 mengabdi 1 to serv

None 76200 abuh II noisy; busy.

None 76200 abui > ABOI.

None 76200 abuk I dust.

None 76200 – bunga pollen.

None 76200 – gergaji sawdust.

None 76200 abuk II (M) hair.

-153035 228600 abuk III (Jv) mengabuk to take someone else’s property by claiming that it belongs to you.

None 76200 abuk-abuk kué ~ kind(s) of cake made from sugar­palm ¶our.

None 76200 abulhayat (A) the father of life, * rain.

None 76200 abun-abun (ob) thoughts, ideas, ideals; > ANGAN­ANGAN.

None 76200 gila di –

None None to hope for something impossible.

None 76200 abunemén > ABONEMÉN.

None 76200 karcis – (bus) commuter ticket.

-152400 229235 abung 1 (Mal) pomelo.

-152400 229235 2 the part of the coconut embryo that grows in the shell.

None None 

None None abur	4

None None 

None 0 

-76200 152400 abur (cla) wasteful, extravagant, prodigal.

-76200 152400 mengabur(kan) to waste, squander (money).

-76200 152400 pengabur spendthrift.

-153035 228600 abus I 1 (col) 1 coins of the lowest denomination.

-1

76200 76200 acung III amaranth (color).

None 76200 ad I (L) – hoc ad hoc.

None 76200 ad II introduces each item in a list.

None 76200 ad III (in acronyms) > ANGKATAN Darat, administrasi.

None 76200 a.d.

None 76200 IV (abbr) [atas dasar] at, on the basis of.

None 76200 AD V (init) > ANGKATAN Darat.

None 76200 AD VI (init) > ANGGARAN Dasar.

None 76200 AD VII car license plate for Surakarta.

-152400 228600 ada 1 there is, there are.

-152400 228600 – buku di atas méja.

-152400 228600 There is/are a book/ books on the table.

-152400 228600 –, tapi tidak banyak.

-152400 228600 There are some, but not many.

-152400 228600 Hampir tidak –.

-152400 228600 There are hardly any.

-152400 228600 – apa? What’s up? What’s the matter? What’s going on? – maksud there’s something he wants.

-152400 228600 – masanya sometimes.

-152400 228600 – orang (this seat’s) occupied, taken.

-152400 228600 – sedia available.

-152400 228600 – sambungannya (at end of a story) to be continued.

-15240

None 227965 ~ perundingan to enter into negotiations.

None 227965 ~ pesanan to place an order.

None 227965 ~ révolusi to bring about a revo<lution.

None 227965 ~ salat to say one’s (obligatory) prayers.

None 227965 ~ sénsasi to

None 0 


None 278765 cause a sensation.

None 278765 3 to operate, run, maintain.

None 278765 ~ daftar umum to maintain public records.

None 278765 ~ jaringan agén-agén to run a spy ring.

None 278765 4 to get, procure, obtain.

None 202565 terada (sifat) ~ (saja) for want of something better.

-76200 278765 keadaan 1 situation, state, nature.

-76200 278765 kata ~ (gram) adjective.

-76200 278765 dapat menguasai ~ to be able to keep the situation under control.

-76200 278765 2 conditions, circumstances.

-76200 278765 dalam ~ demikian under such cir<cumstances.

-76200 278765 dalam ~nya sekarang the way things are now, as is.

-76200 278765 ~ badan physical condition.

-76200 278765 ~ bahaya emergency.

-76200 278765 ~ ber- hati-hati deliberation.

-76

...

None 153035 beratur arranged/placed in proper order, orderly, neatly arranged.

None 229235 ~ baik-baik well­arranged.

None 229235 tidak ~ disorderly, messy.

None 229235 ketidak- beraturan disorder, mess, untidiness.

-76200 229235 mengatur [and ngatur (coq) and ngaturin (J coq)] 1 to arrange, put in order.

-76200 229235 cara ~ pakaian the way one dresses.

-76200 229235 ~ kembali jadwal rescheduling.

-76200 229235 2 to regulate (something legally), govern (of a regulation/clause in a contract).

-76200 229235 Peraturan di bidang lingkungan itu ~ ketentuan bahwa limbah tidak boléh dibuang ke laut.

None 152400 
The regulations in the environmental ¤eld govern the provi<sions that waste may not be disposed of in the sea.

None 152400 3 to stipu<late ( provisions of the law).

None 152400 4 to arrange (for), take care of something, settle.

None 152400 5 to look after, take care of someone 6 to do (one’s hair).

None 152400 7 to string, thread.

None 152400 bisa diatur it can be arra

None 151765 mengaum-(ng)aumkan to boast of something

None 151765 auman roaring.

None 151765 ~ harimau the roaring of tigers.

None 75565 aung (onom) sound of roaring/moaning.

None 151765 mengaung to roar.

None 151765 aungan roaring, wailing.

None 75565 aungan roofbeam.

None 75565 aur I generic name for many large bamboos; > AWI, BAMBU, BULUH.

None 228600 bagai/sebagai – dengan tebing very attached, devoted; to love

None 228600 eachother (husband and wife).

None 228600 (seperti) – ditanam, betung tumbuh to get a windfall.

None 228600 (seperti) – ditarik sungsang very dif¤cult.

None 228600 sebagai

0 228600 dengan rebung very intimate (of friendship).

0 228600 – cina thin bam<boo used for making fences, Bambusa nana.

0 228600 – duri thorny bam<boo, Bambusa blumeana.

0 228600 – gading yellow/common bamboo, Bambusa vulgaris.

0 228600 – hitam black bamboo, Bambusa nigra.

0 228600 – kuning yellow bamboo, Bambusa aspera.

0 228600 – licin thornless bam<boo.

0 228600 – temiang

-152400 None awar average, damage to a ship or its cargo.

-152400 None – khusus/umum (in marine insurance) particular/general average.

-76200 152400 awaracun detoxi¤ed.

-76200 152400 mengawaracunkan to detoxify.

-76200 152400 pengawaracunan detoxi¤cation.

-152400 228600 awar-awar (Jv) a small tree, Ficus septica, whose roots are chewed as antidote to poisonous ¤sh or crabs.

None 12065 awarésin pengawarésinan deresining.

-152400 227965 awas 1 good, sharp (of one’s eyesight).

-152400 227965 Matanya masih –.

-152400 227965 He still has good eyesight.

-152400 227965 kurang – negligent.

-152400 227965 mata kurang – to have weak eyes.

-152400 227965 2 to have the power to foretell the future or the un<known by occult means, be clairvoyant.

-152400 227965 3 (word of caution) beware (of ); watch out for, be on one’s guard against.

-152400 227965 – ada mobil! Watch out for the car! – Anjing! (on signboards) Beware of the dog! – copét! Watch out for pickpockets.

-152400 227965 –! 

0 None – pungut a) adoptive father.

0 None b) ritual father (for certain ceremonies).

0 None – rumah tangga househusband.

-67945 295275 tiri stepfather.

-76835 227965 berayah 1 to have a father.

-76835 227965 2 ~ kepada to call/consider someone one’s father.

None 151765 berayahkan _ to have _ as one’s father.

None 151765 mengayahi to father.

None 151765 keayahan fatherhood, paternity.

None 151765 perayahan paternal.

None 76200 ayah II > AYAT I.

None 76200 ayah III (Hind) Indian nurse; cp AMAH.

-152400 None ayahan (Jv) task, assignment, duty, responsibility.

-152400 None – Umum Public Works (in Yogyakarta); > PEKERJAAN umum.

-152400 None ayahanda (more re¤ned than ayah; usually used in correspon<dence and older literature) father.

None 76200 ayak I a sieve.

None 152400 mengayak(i) to sift, screen.

None 152400 ayakan 1 the result/output of sifting/screening.

None 152400 2 sieve, screen.

None 228600 ~ getar vibrating sieve.

None 228600 ~ kawat wire screen.

-76200 None

None 152400 keazaban torture.

None 76200 azad (Pers) without fault.

None 76200 azahar (A?) hazard.

None 76200 – pekerjaan occupational hazard.

None 76200 azal I (A) without beginning, past without beginning.

None 76200 azal II > AJAL.

None 635 azali (A) 1 of old, from time immemorial, ancient.

None 635 masa/zaman –

None 50800 time immemorial, ancient times.

None 50800 2 punishment after death.

-152400 None azam I (A) greatly respected.

-152400 None isim al.– a greatly respected name (the name of God).

None 76200 azam II (A) intention, purpose, resolution.

None 152400 berazam 1 with intentions.

None 152400 2 resolute, determined.

None 152400 mengazamkan 1 to intend.

None 152400 2 to resolve, make up one’s mind.

None 152400 keazaman intention.

None 76200 azamat (ob) > AZMAT.

None 76200 azan (A) summons/calls by the muezzin for ritual prayers.

None 152400 mengazami to call someone to prayer.

-76200 None mengazani to recite the call to prayer.

-76200 None ~ bayi to re

In [4]:
# Target yang di crawling
# [lema, sublema, gabungan(-,~), similar(panah,"and"), keterangan, akronim([]), terjemahan]

# CARA NGEDAPETIN (bantu isi cara ngedapetin atau ciri khas atau langkah2 buat ngedapetin datanya)
# [v] == sudah dibuat codingannya

# [v] LEMA : indent pertama,bold (simpan di tempLema)
# [v] LEMA (jika sebaris dg lema lainnya) : indent pertama, jika kondisi tempLema isi, jika tidak bisa distemming dan huruf awal kata sama seperti awal kata tempLema
# [v] LEMA (jika sebaris dg sublema) : indent kedua, jika kondisi tempSublema harus isi, jika tidak bisa distemming dan huruf awal kata sama seperti awal kata tempSublema

# [v] SUBLEMA : indent kedua,deteksi bold (simpan di tempSublema)
# [v] SUBLEMA (jika sebaris dg lema): indent pertama, jika kondisi tempLema isi, jika bisa distemming dan huruf awal kata tidak sama seperti awal kata tempLema
# [v] SUBLEMA (jika sebaris dg sublema lainnya) : indent kedua, jika kondisi tempSublema harus isi, jika bisa distemming dan huruf awal kata tidak sama seperti awal kata tempSublema

# [v] GABUNGAN(-,~) : deteksi kata italic, ambil kata-katanya simpan ke list tempGabungan, cek di list ada - atau ~ tidak, kalau ada berarti itu kata gabungan dan replace - dan ~ dengan lema/sublema 

# [v] SIMILAR ("and") : indent pertama/kedua, deteksi bold, jika kata sebelumnya adalah and maka similar
# SIMILAR (panah) :

# [v] KETERANGAN :

# [v] AKRONIM :

# TERJEMAHAN :

In [5]:
# Perisiapan Word :
# - ubah panah jadi >
# - –/~ yang reguler dan bold dijadikan italic
# - ubah font, kemudian tanda yang tidak terbaca seperti di kata "major" (-^w) diganti menjadi <
# - tanda yang tidak terbaca seperti di kata "brother-in-law" (-) diganti menjadi ^w
# - tanda "- " diganti menjadi "<", contoh me- ngelabui (PERTIMBANGKAN LAGI)
# - ubah tanda "..." menjadi "_" (garis bawah)
# - s.t. = something
# - s.o.'s = someone's
# - s.o. = someone
# - q.v. = which see (refer to that word)
# - k.o. = kind(s) of
# - o.s. = oneself
# - e.o. = eachother
# - replace .^w jadi .^p
# - menghapus setiap header halaman dan garisnya



In [7]:
def cleanWord(word) :
    #cek spasi pada kata (jika terdiri dari 2 kata)
    if re.search("\s", word) :
        word = word.split()

#         word[:] = [w for w in word if not (w in roman_numerals or re.search("\d", w) or word == "/" or w == "." or w == "(")]
        
        #update 1
        word[:] = [w for w in word if not (w in roman_numerals or re.search("\d", w))]
    
        word = " ".join(word)
        
        if len(word) < 1 :
            word = None

    #jika terdiri dari 1 kata
    else :
        #update 3
#         if word in roman_numerals or re.search("\d", word) or word == "/" or word == "." or word == "(":
        
        if word in roman_numerals or re.search("\d", word) :
            word = None
    
    return(word)

def replaceData (x) :
    for idx in range(len(x)):
        x[idx] = re.sub("¤","fi",x[idx])
        x[idx] = re.sub("¶","fl",x[idx])
        x[idx] = re.sub("\s{2}|\s{3}"," ",x[idx])
        x[idx] = re.sub("\s*$|\xad|\xad\s*","",x[idx])
        x[idx] = re.sub("^\)|^\)\s*","",x[idx])
        x[idx] = re.sub("\(\s\)","",x[idx])
        x[idx] = re.sub("^\w\)\s*|\s\w\)$","",x[idx])
        x[idx] = re.sub("^\s*|^\s|^\s\w","",x[idx])
        x[idx] = re.sub("\[and","",x[idx])
        x[idx] = re.sub("\[|\[\s","",x[idx])
        x[idx] = re.sub("\]","",x[idx])
        x[idx] = re.sub("\s-\s|-\s","-",x[idx])
        x[idx] = re.sub("\s<\s|\s<|<\s|<","",x[idx])
        x[idx] = re.sub("^\s","",x[idx])
        x[idx] = re.sub("\s\)",")",x[idx])
        x[idx] = re.sub("\(\s","(",x[idx])
        x[idx] = re.sub("^/.*/","",x[idx])
        x[idx] = re.sub("/\s","/",x[idx])
        x[idx] = re.sub("\snya","nya",x[idx])
        x[idx] = re.sub("\skan$","kan",x[idx])
        x[idx] = re.sub("\skan\s","kan ",x[idx])
        
#         x[idx] = re.sub("s\.t\.|s\.t","something",x[idx])
#         x[idx] = re.sub("s\.o\.'s","someone's",x[idx])
#         x[idx] = re.sub("s\.o\.","someone",x[idx])
#         x[idx] = re.sub("q\.v\.","which see (refer to that word)",x[idx])
#         x[idx] = re.sub("k\.o\.","kind(s) of",x[idx])
#         x[idx] = re.sub("o\.s\.","oneself",x[idx])
#         x[idx] = re.sub("e\.o\.","eachother",x[idx])
#         x[idx] = re.sub(";|;\s","",x[idx])

        x[idx] = re.sub("\*\*","e.g.",x[idx])
        x[idx] = re.sub("\*","i.e.,",x[idx])
        x[idx] = re.sub("\.$|\s\.$|\.\s$|\.\s","",x[idx])
#         x[idx] = re.sub("_","...",x[idx])
    
    return x

# fungsi untuk menyimpan hasil ke bentuk list
def cleanData(lema,sublema,gabungan,similar,panah,akronim,antonim,perbandingan,keterangan,terjemahan):
    x = [lema,sublema,gabungan,similar,panah,akronim,antonim,perbandingan,keterangan,terjemahan]
    
    x = replaceData(x)
    
    #split panah
    if x[4] != "" :
        x[4] = re.split(",|;",x[4])
    
    #split terjemahan
    if x[8] != "" :
        x[8] = re.split(",|;",x[8])
        x[8] = replaceData(x[8])
        
        if "" in x[8] :
            x[8].remove("")
        
        for idx in range(len(x[8])):
            x[8][idx] = re.sub("_","...",x[8][idx])
            
        if len(x[8]) < 1 :
            x[8] = ""
    
    y = pd.DataFrame(data=[x],columns=["lema","sublema","gabungan","similar","panah","akronim","antonim","perbandingan","keterangan","terjemahan"])
    
    return x,y

def cleanParenthesis(word) :
    if '(' in word :
        x, y = re.sub(r"[/(].*?[/))=]","",word),re.sub(r"[()=]","",word)
        x, y = re.sub(r"\s$","",x),re.sub(r"\s$","",y)
        word = x
        similar = y
        tempLemaSublema = word
        boolLemaSublema = True
    else :
        word = re.sub(r"[()]|\s$","",word)
        similar = ""
        tempLemaSublema = word
        boolLemaSublema = True
        
    return word,similar,tempLemaSublema,boolLemaSublema

def readItalicData(word,index,paragraph,keterangan,tempKeterangan,tempGabungan,terjemahan) :     
    if word == "opp" or word == "cp":
        word = ""

    #KETERANGAN
    # mengambil keterangan
    if index > 0 and re.search("\(", paragraph.runs[index-1].text):

        if re.search("\s", word) :
            split_text = word.split()
            tempWord = []

            for s in split_text :
                if s in list_ket :
                    tempWord.append(s)
                else :
                    break

            word = " ".join(tempWord)

            tempKeterangan.append(word)

        elif word in list_ket :
            tempKeterangan.append(word)
            
        else :
            #update 2
            if paragraph.runs[index+1].italic or re.search("\)", paragraph.runs[index+1].text) :
                tempGabungan.append(word)
            else :
                if terjemahan == "" and word not in roman_numerals and boolPanah != True:
                    terjemahan = word
                elif terjemahan != "" and word not in roman_numerals and boolPanah != True:
                    terjemahan = terjemahan+" "+word

        # update sementara
        try :
            if re.search("\)", word) or re.search("\)", paragraph.runs[index+1].text) :
                keterangan = " ".join(tempKeterangan)
                tempKeterangan = []
        except :
            if re.search("\)", word) :
                keterangan = " ".join(tempKeterangan)
                tempKeterangan = []
        
#         keterangan = "satu"

    # mengambil keterangan selanjutnya jika terdiri dari > 1 keterangan
    elif len(tempKeterangan) > 0 and word not in tempKeterangan:
        if re.search("\s", word) :
            split_text = word.split()
            tempWord = []

            for s in split_text :
                if s in list_ket :
                    tempWord.append(s)
                else :
                    break

            word = " ".join(tempWord)

            tempKeterangan.append(word)

        elif word in list_ket :
            tempKeterangan.append(word)

            try :
                if re.search("\)", paragraph.runs[index+1].text) or re.search("\)", word) :
                    keterangan = " ".join(tempKeterangan)
                    tempKeterangan = []
            except :
                print("terjadi kesalahan")
                
        else :
            if terjemahan == "" and word not in roman_numerals and boolPanah != True:
                terjemahan = word
            elif terjemahan != "" and word not in roman_numerals and boolPanah != True:
                terjemahan = terjemahan+" "+word
#         keterangan = "dua"

    else :
        tempGabungan.append(word)
#         keterangan = "tiga"
    
    return keterangan,tempKeterangan,tempGabungan,terjemahan

In [8]:
lexicon = pd.DataFrame(columns=["lema","sublema","gabungan","similar","panah","akronim","antonim","perbandingan","keterangan","terjemahan"])
roman_numerals = ["I","II","III","IV","V","VI","VII","VIII","IX","X"]
list_and = ["and", " and", "and ", " and ", "[and", "[and "]
list_ket = ["A","abbr","Ac","acr","adj","AE","anat","app","Bal","Ban","Bat","BD","BE","BG","bio","bot","C","chem","cla","col","coq","cp","D","D/E","derog","E","elec","e.o.","epist","esp","euph","exclam","fin","Fr","G","geo","geol","Gr","gram","Hind","IBT","infr","init","insur","Irja","Isl","J","J/Jv","joc","Jp","Jv","K","k.o.","L","leg","ling","lit","M","Mad","Mal","math","Med","med","mil","Min","mod","mus","naut","NTB","NTT","ob","obj","O jv","onom","opp","o.s.","Pal","Pap","Pers","petro","phys","pl","pl obj","pl subj","poet","Port","Pr","pron","q.v.","RC","reg","rel","rev","S","sg","Sg","Skr","Skr neo","sl","s.o.","s.o.'s","s.t.","stat","subj","Sum","Tag","Tam","usu","voc","vulg","zod"]

#variabel
lema = ""
sublema = ""
gabungan = ""
similar = ""
panah = ""
akronim = ""
antonim = ""
perbandingan = ""
keterangan = ""
terjemahan = ""

boolLemaSublema = False
boolPanah = False
tempLema = ""
tempSublema = ""
# tempLemaSublema = ""
tempTerjemahan = ""
tempGabungan = []
tempKeterangan = []
tempAkronim = []
    
index = 0

#looping paragraf tiap document
for paragraph in document.paragraphs:
    
    boolLemaSublema = False
    
    index = 0
    
    #menyimpan nilai indentasi (first indent dan left indent)
    first = paragraph.paragraph_format.first_line_indent
    left = paragraph.paragraph_format.left_indent
    
    #looping setiap kata di setiap paragrafnya
    for run in paragraph.runs:
        
#         print("word = "+run.text)
#         print("index = "+str(index))
#         print("length = "+str(len(paragraph.runs)))
        
        #mengatasi left indent == none, dengan mengubah nilai variabel left dengan tempLeft (nilai pada left sebelumnya)
        if left == None :
            left = 228600 
        
        #deteksi kata bold
        if run.bold and run.text.isspace() == False :
            if run.text.isspace() == False :
                #baca indent lema
                if (((first == None or first == 0 or first == 76200) and (left <= 76835)) or ((first == -152400 or first == -153035) and left >= 227965)) :      

                    tempLeft = left

                    if lema != "" or sublema != "" or gabungan != "" or similar != "" or panah != "" or akronim != "" or antonim!= "" or keterangan != "" or terjemahan != "":
                        x = cleanData(lema,sublema,gabungan,similar,panah,akronim,antonim,perbandingan,keterangan,terjemahan)
                        print(x[0])
                        lexicon  = lexicon.append([x[1]],ignore_index=True)

                    #variabel 
                    lema = ""
                    sublema = ""
                    gabungan = ""
                    similar = ""
                    panah = ""
                    akronim = ""
                    antonim = ""
                    perbandingan = ""
                    keterangan = ""
                    terjemahan = ""

                    boolPanah = False
                    tempGabungan = []
                    tempKeterangan = []
                    tempAkronim = []

                    word = cleanWord(run.text)

                    if word != None :
                        
#                         print("run.text = "+run.text)
#                         print(word)
                        
                        #LEMA
                        #jika lema kosong atau belum terdapat lema, lema = word
                        if boolLemaSublema == False :

                            results = cleanParenthesis(word)

                            lema = results[0]
                            similar = results[1]
#                             tempLemaSublema = results[2]
                            boolLemaSublema = [3]
                            
                            tempLema = lema

                        #jika lema tidak kosong atau variabel lema sudah terisi
                        else :
#                             print("prev = "+paragraph.runs[index-1].text)
#                             print("word = "+word)
                            
                            #SIMILAR (AND)
                            #jika kata sebelum kata style bold adalah "and" maka dianggap sebagai similar
                            if paragraph.runs[index-1].text in list_and or paragraph.runs[index-2].text in list_and :
#                                 print(word)
                                similar = word

                            #jika bukan similar
                            else :  
                                #SUBLEMA(JIKA SEBARIS/SEPARAGRAF DENGAN LEMA)  
                                #jika sublema kosong
                                if sublema == "" :
                                    if lema != "" or sublema != "" or gabungan != "" or similar != "" or panah != "" or akronim != "" or antonim!= "" or keterangan != "" or terjemahan != "":
                                        x = cleanData(lema,sublema,gabungan,similar,panah,akronim,antonim,perbandingan,keterangan,terjemahan)
                                        print(x[0])
                                        lexicon  = lexicon.append([x[1]],ignore_index=True)

                                    lema = ""
                                    similar = ""
                                    gabungan = ""
                                    similar = ""
                                    panah = ""
                                    akronim = ""
                                    antonim = ""
                                    perbandingan = ""
                                    keterangan = ""
                                    terjemahan = ""

                                    results = cleanParenthesis(word)

                                    sublema = results[0]
                                    similar = results[1]
#                                     tempLemaSublema = results[2]
                                    tempSublema = results[2]
                                    boolLemaSublema = [3]

                                #jika sublema tidak kosong, maka simpan sublema ke list terlebih dahulu
                                else :
                                    if lema != "" or sublema != "" or gabungan != "" or similar != "" or panah != "" or akronim != "" or antonim!= "" or keterangan != "" or terjemahan != "":
                                        x = cleanData(lema,sublema,gabungan,similar,panah,akronim,antonim,perbandingan,keterangan,terjemahan)
                                        print(x[0])
                                        lexicon  = lexicon.append([x[1]],ignore_index=True)

                                    lema = ""
                                    similar = ""
                                    gabungan = ""
                                    similar = ""
                                    panah = ""
                                    akronim = ""
                                    antonim = ""
                                    perbandingan = ""
                                    keterangan = ""
                                    terjemahan = ""

                                    results = cleanParenthesis(word)

                                    sublema = results[0]
                                    similar = results[1]
#                                     tempLemaSublema = results[2]
                                    tempSublema = results[2]
                                    boolLemaSublema = [3]

                #baca indent sublema
                else :
                    tempLeft = left 

                    #deteksi kata bold
                    if(run.bold) and run.text.isspace() == False :

                        if lema != "" or sublema != "" or gabungan != "" or similar != "" or panah != "" or akronim != "" or antonim!= "" or keterangan != "" or terjemahan != "":
                            x = cleanData(lema,sublema,gabungan,similar,panah,akronim,antonim,perbandingan,keterangan,terjemahan)
                            print(x[0])
                            lexicon  = lexicon.append([x[1]],ignore_index=True)

                        #variabel
                        lema = ""
                        sublema = ""
                        gabungan = ""
                        similar = ""
                        panah = ""
                        akronim = ""
                        antonim = ""
                        perbandingan = ""
                        keterangan = ""
                        terjemahan = ""

                        boolPanah = False
                        tempGabungan = []
                        tempKeterangan = []
                        tempAkronim = []

                        word = cleanWord(run.text)

                        if word != None :
                            
#                             print("prev = "+paragraph.runs[index-1].text)
#                             print("word = "+word)
                            
                            #SUBLEMA
                            #jika sublema kosong
                            if boolLemaSublema == False :

                                results = cleanParenthesis(word)

                                sublema = results[0]
                                similar = results[1]
#                                 tempLemaSublema = results[2]
                                tempSublema = results[2]
                                boolLemaSublema = [3]

                            #jika sublema tidak kosong
                            else :
                                #SIMILAR (AND)
                                #jika kata sebelum kata style bold adalah "and" maka dianggap sebagai similar
                                if paragraph.runs[index-1].text in list_and or paragraph.runs[index-2].text in list_and :
                                    similar = word

                                #SUBLEMA (JIKA SUBLEMA SEBARIS/SEPARAGRAF DENGAN SUBLEMA YANG LAIN)
                                #jika bukan, maka simpan sublema ke list terlebih dahulu
                                else :
                                    if lema != "" or sublema != "" or gabungan != "" or similar != "" or panah != "" or akronim != "" or antonim!= "" or keterangan != "" or terjemahan != "":
                                        x = cleanData(lema,sublema,gabungan,similar,panah,akronim,antonim,perbandingan,keterangan,terjemahan)
                                        print(x[0])
                                        lexicon  = lexicon.append([x[1]],ignore_index=True)

                                    lema = ""
                                    sublema = ""
                                    gabungan = ""
                                    similar = ""
                                    panah = ""
                                    akronim = ""
                                    antonim = ""
                                    perbandingan = ""
                                    keterangan = ""
                                    terjemahan = ""

                                    results = cleanParenthesis(word)

                                    sublema = results[0]
                                    similar = results[1]
#                                     tempLemaSublema = results[2]
                                    tempSublema = results[2]
                                    boolLemaSublema = [3]

        #deteksi kata italic (kata gabungan, keterangan)
#             print("sublema = "+sublema)
#             print("similar = "+similar)
#             print("tempLemaSublema = "+tempLemaSublema)
            
        elif run.italic :
            if run.text.isspace() == False :
                results = readItalicData(run.text,index,paragraph,keterangan,tempKeterangan,tempGabungan,terjemahan)

#                 print("prev = "+paragraph.runs[index-1].text)
#                 print(paragraph.runs[index-1].italic)
#                 print("text italic = "+run.text)
                
                keterangan = results[0]
                tempKeterangan = results[1]
                tempGabungan = results[2] 
                terjemahan = results[3]
                
#                 print(results)
                
        # deteksi kata reguler (akronim, terjemahan)
        else : 
            if run.text.isspace() == False :
                word = run.text

                if word != None :
                    #GABUNGAN
#                     if "–" in tempGabungan or "~" in tempGabungan:
#                         gabungan = " ".join(tempGabungan)
#                         gabungan = re.sub("–|~",tempLemaSublema,gabungan)
#                         gabungan = re.sub("  "," ", gabungan)

#                         results = cleanParenthesis(gabungan)

#                         gabungan = results[0]
#                         similar = results[1]

#                         tempGabungan = []

#                     print(tempGabungan)

                    if "–" in tempGabungan:
                        gabungan = " ".join(tempGabungan)
                        gabungan = re.sub("–",tempLema,gabungan)
                        gabungan = re.sub("  "," ", gabungan)

                        results = cleanParenthesis(gabungan)
                        
                        gabungan = results[0]
                        similar = results[1]
                        
                        if lema != "" or sublema != "" or gabungan != "" or similar != "" or panah != "" or akronim != "" or antonim!= "" or keterangan != "" or terjemahan != "":
                            x = cleanData(lema,sublema,gabungan,similar,panah,akronim,antonim,perbandingan,keterangan,terjemahan)

                            print(x[0])
                            lexicon  = lexicon.append([x[1]],ignore_index=True)

                        lema = ""
                        sublema = ""
                        gabungan = ""
                        similar = ""
                        panah = ""
                        akronim = ""
                        antonim = ""
                        perbandingan = ""
                        keterangan = ""
                        terjemahan = ""
                            
                        boolPanah = False
                        tempGabungan = []
                        tempKeterangan = []
                        tempAkronim = []

#                         gabungan = results[0]
#                         similar = results[1]

#                         tempGabungan = []
                    
                    elif "~" in tempGabungan:
                        gabungan = " ".join(tempGabungan)
                        gabungan = re.sub("~",tempSublema,gabungan)
                        gabungan = re.sub("  "," ", gabungan)

                        results = cleanParenthesis(gabungan)
                        
                        gabungan = results[0]
                        similar = results[1]
                        
                        if lema != "" or sublema != "" or gabungan != "" or similar != "" or panah != "" or akronim != "" or antonim!= "" or keterangan != "" or terjemahan != "":
                            x = cleanData(lema,sublema,gabungan,similar,panah,akronim,antonim,perbandingan,keterangan,terjemahan)

                            print(x[0])
                            lexicon  = lexicon.append([x[1]],ignore_index=True)

                        lema = ""
                        sublema = ""
                        gabungan = ""
                        similar = ""
                        panah = ""
                        akronim = ""
                        antonim = ""
                        perbandingan = ""
                        keterangan = ""
                        terjemahan = ""
                            
                        boolPanah = False
                        tempGabungan = []
                        tempKeterangan = []
                        tempAkronim = []

#                         gabungan = results[0]
#                         similar = results[1]

#                         tempGabungan = []

                    else :
                        if len(tempGabungan) != 0:
                            
#                             print ("length = "+str(len(tempGabungan)))
#                             print ("index = "+str(index))
#                             print(tempGabungan)
#                             print("tempLema = "+tempLema )

                            temp = " ".join(tempGabungan)
    
#                             print("temp = "+temp)
                            
                            if boolPanah == True and panah != word and word not in roman_numerals :
#                                 temp = " ".join(tempGabungan)
                                panah = panah+" "+temp

                                temp = ""
                                tempGabungan = []
                            
#                             elif len(tempGabungan) > 1 and (tempLema.lower() in temp.lower() or tempLemaSublema.lower() in temp.lower()) :
                            elif len(tempGabungan) > 1 and (tempLema.lower() in temp.lower() or tempSublema.lower() in temp.lower()) :
#                                 temp = " ".join(tempGabungan)
                                gabungan = gabungan+" "+temp
                                gabungan = re.sub(r'  ',' ', gabungan)

                                results = cleanParenthesis(gabungan)

                                gabungan = results[0]
                                similar = results[1]

                                tempGabungan = []
                            
                            elif index > 1 and len(tempGabungan) <= 3:
#                                 temp = " ".join(tempGabungan)
                                terjemahan = terjemahan+" "+temp

                                temp = ""
                                tempGabungan = []

                            else :
#                                 temp = " ".join(tempGabungan)
                                gabungan = gabungan+" "+temp
                                gabungan = re.sub(r'  ',' ', gabungan)

                                results = cleanParenthesis(gabungan)

                                gabungan = results[0]
                                similar = results[1]

                                tempGabungan = []

                #AKRONIM
                # mengambil akronim
                if re.search("\[", paragraph.runs[index-1].text) and word not in list_and:
                    tempAkronim.append(word)

                    if re.search("\]", paragraph.runs[index+1].text) :
                        akronim = " ".join(tempAkronim)
                        tempAkronim = []

                # mengambil akronim selanjutnya jika terdiri > 1 kata
                elif len(tempAkronim) > 0 and word not in tempAkronim :
                    tempAkronim.append(word)

                    try :
                        if re.search("\]", paragraph.runs[index+1].text) :
                            akronim = " ".join(tempAkronim)
                            akronim = re.sub(r'  ',' ', akronim)
                            tempAkronim = []
                    except :
                        if re.search("\]", paragraph.runs[index].text) :
                            akronim = " ".join(tempAkronim)
                            akronim = re.sub(r'  ',' ', akronim)
                            tempAkronim = []
                else :
                    #PANAH   
                    
#                     print("prev-1 = "+paragraph.runs[index-1].text)
                    
#                     if index > 1 :
#                         print("prev-2 = "+paragraph.runs[index-2].text)
                        
#                     print("word = "+word)
#                     print("terjemahan = "+terjemahan)
#                     print("index = "+str(index))
#                     print("length = "+str(len(paragraph.runs)))
                    
                    if boolPanah == True and word not in roman_numerals :
                        tempWord = re.sub("\.","", word)
                        tempWord = cleanWord(tempWord)
                     
#                         print("word 2= "+word)
#                         print(tempWord)

                        if tempWord != None :
                            panah = panah+" "+tempWord
                            word = ""
                            
#                         print("panah 2 = "+panah)
                    
                    if boolPanah == True and re.search("\.", word) :
#                         print("word 3= "+word)
                        
                        panah = re.sub("\.|\s\.","",panah)
                        boolPanah = False
                        word = ""
#                         print("panah 3 = "+panah)
                    
                    
#                     if word != "" and word != "." and (paragraph.runs[index-1].text == ">" or (index > 1 and paragraph.runs[index-2].text == ">")) :
                    if word != "" and word != "." and (re.search(">", paragraph.runs[index-1].text) or (index > 1 and re.search(">", paragraph.runs[index-2].text) )) :

#                         print("run.text = "+run.text)
#                         print("word 1= "+word)

                        word = re.sub("\.","", word)
                        word = cleanWord(word)

                        if word != None :
                            panah = word
                            boolPanah = True
                            word = ""
                            
#                         print("panah = "+panah)
                        
#                         print("panah 1 = "+panah)
#                             
#                             try :
#                                 print(paragraph.runs[index+1].text)
#                                 print(panah)
#                             except :
#                                 print(paragraph.runs[index+0].text)
#                                 print(panah)

                    elif paragraph.runs[index-1].italic == True and (paragraph.runs[index-1].text == "opp" or (index > 1 and paragraph.runs[index-2].text == "opp")):
                        re.sub("\s","",word)

                        antonim = word
                
                    elif paragraph.runs[index-1].italic == True and (paragraph.runs[index-1].text == "cp" or (index > 1 and paragraph.runs[index-2].text == "cp")):
                        re.sub("\s","",word)

                        perbandingan = word

                    else :

                        #TERJEMAHAN
                        if re.search("\]\s", word) :
                            word = re.sub("\]\s","",word)

                        if word in list_and and (paragraph.runs[index-1].bold == True and (paragraph.runs[index+1].bold == True or paragraph.runs[index+2].bold == True)):
                            word = ""

                        if word == "[" or word == "]" or re.search(">", word) :
                            word = ""

#                         if terjemahan == "" and word not in roman_numerals and boolPanah != True :
                        if terjemahan == "" and boolPanah != True :
                            terjemahan = word
#                         elif terjemahan != "" and word not in roman_numerals and boolPanah != True:
                        elif terjemahan != "" and boolPanah != True:
                            terjemahan = terjemahan+" "+word

#                         print("prev = "+paragraph.runs[index-1].text)
#                         print("word = "+word)
#                         print("terjemahan = "+terjemahan)
#                         print("run.text = "+run.text)
                        if re.search("\.|\;|\!|\?", run.text) or re.match("\w\)", run.text) :
        
#                             print("hihi")
            
                            if lema != "" or sublema != "" or gabungan != "" or similar != "" or panah != "" or akronim != "" or antonim!= "" or keterangan != "" or terjemahan != "":
                                x = cleanData(lema,sublema,gabungan,similar,panah,akronim,antonim,perbandingan,keterangan,terjemahan)

                                print(x[0])
                                lexicon  = lexicon.append([x[1]],ignore_index=True)

                            lema = ""
                            sublema = ""
                            gabungan = ""
                            similar = ""
                            panah = ""
                            akronim = ""
                            antonim = ""
                            perbandingan = ""
                            keterangan = ""
                            terjemahan = ""
                            
                            boolPanah = False
                            tempGabungan = []
                            tempKeterangan = []
                            tempAkronim = []
                            
#                             boolPanah = False
        index += 1                 

['', '', '', '', '', '', '', '', '', 'A']
['a', '', '', '', '', '', '', '', '', '']
['', '', '', 'A', '', '', '', '', '', '']
['', '/', '', '', '', '', '', '', '', 'a/the first letter of the Latin alphabet used for writing Indonesian']
['a', '', '', '', '', '', '', '', '', '']
['', '', '', '', '', '', '', '', ['abbr'], 'are (a unit of square measure in the metric system, equal to 100 square meters or about 119.6']
['', '', '', '', '', '', '', '', '', 'square yards)']
['a', '', '', '', '', '', '', '', ['abbr'], 'ampere']
['a', '', '', '', '', '', '', '', '', 'ahh (hesitation)']
['A', '', '', '', '', '', '', '', '', '']
['', '', '', '', '', '', '', '', '', '(in music) the sixth tone/note in the ascending scale of C major']
['A', '', '', '', '', '', '', '', '', 'car license plate for Banten']
['à', '', '', '', '', '', '', '', '', '']
['', '', '', '@', '', '', '', '', '', 'at']
['', '', 'à Rp 1', '', '', '', '', '', '', '']
['', '', '', '', '', '', '', '', '', '']
['', '', '', '', '', '', 

['', '', '', '', '', '', '', '', '', 'to neglect, ignore, disregard, pay no attention to']
['', '', 'Negara itu berani mengabaikan résolusi itu', '', '', '', '', '', '', '']
['', '', '', '', '', '', '', '', '', '']
['', '', '', '', '', '', '', '', '', 'That country had the nerve to ignore the resolution']
['', '', '', '', '', '', '', '', '', 'to underestimate (the enemy, etc.)']
['', '', 'Jangan abaikan musuh yang kuat itu', '', '', '', '', '', '', '']
['', '', '', '', '', '', '', '', '', 'Don’t underestimate a strong enemy']
['', '', '', '', '', '', '', '', '', 'to break (a promise/agreement), betray (a secret) through carelessness']
['', '', 'mengabaikan janji', '', '', '', '', '', '', '']
['', '', '', '', '', '', '', '', '', 'to break a promise']
['', '', '', '', '', '', '', '', '', 'to overlook (a fact), miss (a chance), not comply with']
['', '', 'Sayang sekali Anda abaikan peluang emas itu', '', '', '', '', '', '', '']
['', '', '', '', '', '', '', '', '', 'It’s a shame you missed

['', '', 'serbuk/obat abaté', '', '', '', '', '', '', '']
['', '', '', '', '', '', '', '', '', 'larvicide powder']
['', 'pengabatéan', '', '', ['ABATISASI'], '', '', '', '', '']
['abatisasi', '', '', '', '', '', '', '', '', 'exterminating mosquito larvae by using abaté']
['abatoar', '', '', '', '', '', '', '', ['E'], 'abattoir']
['abau', '', '', '', '', '', '', '', '', 'kind(s) of large swamp turtle, painted terrapin, Callagur picta']
['abawi', '', '', '', '', '', '', '', ['A'], 'paternal']
['abaya', '', '', '', '', '', '', '', '', 'long black coat worn by women on the pilgrimage to Mecca']
['', 'berabaya', '', '', '', '', '', '', '', 'to wear such a coat']
['abc', '', '', '', '', '', '', '', '', '']
['', '', '', 'ABC', '', '', '', '', '', ' alphabet, the abc’s']
['', '', 'tidak tahu sama sekali abc', '', '', '', '', '', '', '']
['', '', '', '', '', '', '', '', '', 'not to have the slightest idea (about something)']
['abd', '', '', '', ['ABDI'], '', '', '', '', '']
['abdas', '', '', ''

['', '', '', '', '', '', '', '', '', '']
['', '', '', '', '', '', '', '', '', '']
['', '', '', '', '', '', '', '', '', 'ing']
['', '', 'abis ngapain?', '', '', '', '', '', '', '']
['', '', '', '', '', '', '', '', '', 'What have you been doing?']
['', 'ngabisin', '', '', '', '', '', '', '', 'to use up, finish off;']
['', '', '', '', ['MENGHABISKAN'], '', '', '', '', '']
['', 'ngabis-ngabisin', '', '', '', '', '', '', '', 'to use something all up']
['abis', '', '', '', '', '', '', '', ['E'], 'abyss']
['abisal', '', '', '', '', '', '', '', ['E'], '(geo) abyssal']
['abisan', '', '', '', ['habis'], '', '', '', '', '']
['abit', '', '', '', '', '', '', '', ['ob naut'], 'bitts']
['abiturién', '', '', '', '', '', '', '', ['D'], 'high school graduate']
['abiz', '', '', '', ['ABIS'], '', '', '', ['BG'], '']
['abjad', '', '', '', '', '', '', '', ['A'], '']
['', '', '', '', '', '', '', '', '', 'alphabet (such as alif-ba-ta , etc.)']
['', '', 'kaf', '', '', '', '', '', ['ob'], 'the sequence of Arabi

['', 'ngabrukin', '', '', '', '', '', '', '', 'to slam something down']
['', '', 'ngabrukin pantatnya di atas sofa', '', '', '', '', '', '', '']
['', '', '', '', '', '', '', '', '', 'to slam one’s behind down on the sofa']
['ABS', '', '', '', '', 'Asal Bapak Senang', '', '', ['init'], 'Pleasing the Boss;']
['', '', '', '', ['ASAL Bapak Senang'], '', '', '', '', '']
['', 'meng-ABS-i', '', '', '', '', '', '', '', '']
['', '', '', 'meng-ABS-kan', '', '', '', '', '', 'to flatter']
['absah', '', '', '', '', '', '', '', ['A'], 'valid, legal, legitimate;']
['', '', '', '', ['SAH'], '', '', '', '', '']
['', '', 'tidak absah', '', '', '', '', '', '', '']
['', '', '', '', '', '', '', '', '', 'invalid, illegal']
['', '', 'Kontrak ini tidak absah', '', '', '', '', '', '', '']
['', '', '', '', '', '', '', '', '', '']
['', '', '', '', '', '', '', '', '', 'This contract is invalid']
['ketidakabsahan', '', '', '', '', '', '', '', '', 'invalidity, illegality']
['', 'mengabsahkan', '', '', '', '', '', '

...

['', '', '', '', '', '', '', '', '', 'Public Works (in Yogyakarta);']
['', '', '', '', ['PEKERJAAN umum'], '', '', '', '', '']
['ayahanda', '', '', '', '', '', '', '', '', '(more refined than ayah ;']
['', '', '', '', '', '', '', '', '', 'usually used in correspondence and older literature) father']
['ayak', '', '', '', '', '', '', '', '', 'a sieve']
['', 'mengayak', '', 'mengayaki', '', '', '', '', '', 'to sift, screen']
['', 'ayakan', '', '', '', '', '', '', '', '']
['', '', '', '', '', '', '', '', '', 'the result/output of sifting/screening']
['', '', '', '', '', '', '', '', '', 'sieve, screen']
['', '', 'ayakan getar', '', '', '', '', '', '', '']
['', '', '', '', '', '', '', '', '', 'vibrating sieve']
['', '', 'ayakan kawat', '', '', '', '', '', '', '']
['', '', '', '', '', '', '', '', '', 'wire screen']
['', 'pengayak', '', '', '', '', '', '', '', '']
['', '', '', '', '', '', '', '', '', 'the person who sifts']
['', '', '', '', '', '', '', '', '', 'sieve, screen']
['', '', '', '',

['', '', 'ayam goréngan', '', '', '', '', '', '', '']
['', '', '', '', '', '', '', '', '', 'fryer']
['', '', 'ayam goréng mbok Brék', 'ayam goréng mbok Berék', '', '', '', '', '', '']
['', '', '', '', '', '', '', '', '', 'chicken in a basket (simmered in spiced coconut milk) (kind(s) of fast food chicken)']
['', '', 'ayam gulai', '', '', '', '', '', '', '']
['', '', '', '', '', '', '', '', '', 'stewed chicken']
['', '', 'ayam Hainan', '', '', '', '', '', '', '']
['', '', '', '', '', '', '', '', '', 'a way of preparing chicken']
['', '', 'ayam hutan', '', '', '', '', '', '', '']
['', '', '', '', ['AYAM alas'], '', '', '', '', '']
['', '', 'ayam hutan hijau', '', '', '', '', '', '', '']
['', '', '', '', '', '', '', '', '', 'green jungle fowl, Gallus varius']
['', '', 'ayam induk', '', '', '', '', '', '', '']
['', '', '', '', '', '', '', '', '', 'parent stock']
['', '', 'ayam isi', '', '', '', '', '', '', '']
['', '', '', '', '', '', '', '', '', 'dressed and stuffed chicken']
['', '', 'ay

['', '', '', '', '', '', '', '', '', 'redfeathered fighting cock with black eyes and legs']
['', '', 'ayam telur', '', '', '', '', '', '', '']
['', '', '', '', '', '', '', '', '', 'layer (chicken)']
['', '', 'ayam ternak', '', '', '', '', '', '', '']
['', '', '', '', '', '', '', '', '', 'poultry']
['', '', 'ayam teruna', '', '', '', '', '', '', '']
['', '', '', '', '', '', '', '', '', 'cockerel']
['', '', 'ayam tukung', '', '', '', '', '', '', '']
['', '', '', '', '', '', '', '', '', '']
['', '', '', '', '', '', '', '', '', 'featherless chicken']
['', '', '', '', '', '', '', '', '', '']
['', '', '', '', '', '', '', '', '', 'tailless chicken']
['', '', 'ayam tulak', '', '', '', '', '', '', '']
['', '', '', '', '', '', '', '', '', '']
['', '', '', '', '', '', '', '', '', 'chicken with uprightstanding feathers like a porcupine']
['', '', '', '', '', '', '', '', '', '']
['', '', '', '', '', '', '', '', '', 'spotted chicken']
['', 'ayam-ayam', '', '', '', '', '', '', '', 'watercock, wild he

['', 'mengayum', '', '', '', '', '', '', '', 'to abet, furnish with money (for bad purposes)']
['', 'ayuman', '', '', '', '', '', '', '', 'collusion']
['', 'pengayuman', '', '', '', '', '', '', '', 'abetting, colluding']
['ayun', '', '', '', '', '', '', '', '', '']
['', '', '', '', '', '', '', '', '', 'oscillation, swinging back and forth']
['', '', '', '', '', '', '', '', '', 'sway']
['', '', 'dengan ayun dengkul', '', '', '', '', '', '', '']
['', '', '', '', '', '', '', '', '', 'on foot']
['', '', '', '', '', '', '', '', '', 'swing (in boxing)']
['', '', 'ayun ambung', '', '', '', '', '', '', '']
['', '', '', '', '', '', '', '', '', 'swaying']
['mengayun-ambungkan', '', 'mengayun-ambungkan temayun', '', '', '', '', '', '', 'to swing, rock something']
['', '', '', '', '', '', '', '', '', '']
['', '', '', '', '', '', '', '', '', 'to hang down and swing']
['', '', '', '', '', '', '', '', '', '']
['', '', '', '', '', '', '', '', '', 'to set, sink, go down']
['', '', 'matahari mengayun-am

['', '', '', '', '', '', '', '', '', 'resolute, determined']
['', 'mengazamkan', '', '', '', '', '', '', '', '']
['', '', '', '', '', '', '', '', '', 'to intend']
['', '', '', '', '', '', '', '', '', 'to resolve, make up one’s mind']
['', 'keazaman', '', '', '', '', '', '', '', 'intention']
['azamat', '', '', '', ['AZMAT'], '', '', '', ['ob'], '']
['azan', '', '', '', '', '', '', '', ['A'], 'summons/calls by the muezzin for ritual prayers']
['', 'mengazami', '', '', '', '', '', '', '', 'to call someone to prayer']
['', 'mengazani', '', '', '', '', '', '', '', 'to recite the call to prayer']
['', '', 'mengazani bayi', '', '', '', '', '', '', '']
['', '', '', '', '', '', '', '', '', 'to recite the call to prayer over a newborn child']
['', '', 'mengazani mayat', '', '', '', '', '', '', '']
['', '', '', '', '', '', '', '', '', 'to recite the call to prayer over the deceased at funeral rites']
['', 'mengazankan', '', '', '', '', '', '', '', 'to summon/call for the ritual prayer']
['', 'aza

In [9]:
lexicon

,lema,sublema,gabungan,similar,panah,akronim,antonim,perbandingan,keterangan,terjemahan
0,,,,,,,,,,A
1,a,,,,,,,,,
2,,,,A,,,,,,
3,,/,,,,,,,,a/the first letter of the Latin alphabet used ...
4,a,,,,,,,,,
...,...,...,...,...,...,...,...,...,...,...
20609,Aztéc,,,,,,,,[D],Aztec
20610,azza,,,,,,,,[A],"glorious, honored"
20611,,,azza,,,,,,,
20612,,,,,,,,,,


In [10]:
# lexicon.to_excel (r'A.xlsx', index = False, header=True)